# Ajout de contraintes au modèle K=2

In [ ]:
function uncertain_model_2_eff(C_2,V_2,C_v, S, B, logs, relax=false, t_max=120)
    """
    Crée et optimise le programme linéaire avec cycles de taille 2 augmenté de la contrainte "d'efficacité".

    Entrées : 
        - C_2   : array   - Liste des identifiants des cycles de taille 2
        - V_2   : array   - Liste des sommets concernés par les cycles de taille 2
        - C_v   : array   - Liste des identifiants des cycles passant par chaque sommet 
        - S     : array   - Liste des scénarios
        - B     : Integer - Budget (nombre de tests maximum)
        - logs  : Boolean - Booléen pour l'affichage de la solution
        - relax : Boolean - Booléen pour relâcher la contrainte d'intégrité
        - t_max : Integer - Temps maximum accordé pour la résolution du problème
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Vecteur des sommets 
    N = Array(1:length(S))
    
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    ## x[(i,j)] = 1 ssi on réalise un test croisé sur les deux arcs (i,j) et (j,i)    
    @variable(model, x[C_2], Bin);
    ## z[(i,j),s] = 1 ssi le cycle [(i,j),(j,i)] effectue un transfert dans le scénario s
    # Relaxation de z si souhaité
    if relax
        @variable(model, z[C_2,N] >=0);
    else
        @variable(model, z[C_2,N], Bin);
    end
    
    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantations
    @objective(model, Max, 2/length(S)*sum(z[c,s] for c in C_2 for s in N));
    
    # Contraintes
    ## On réalise au plus B tests croisés (B = budget de tests)
    @constraint(model,c1, 2*sum(x[a] for a in C_2) <= B)
    
    ## NOUVELLE CONTRAINTE : On force x[a] a être nul si a n'apparaît dans aucun cycle au sein des N scénarios
    @constraint(model,c1ter[a in C_2], x[a] <= sum(z[a,s] for s in N))
        
    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc (dans le scénario s)
    @constraint(model,c2[c in C_2, s in N],
        2 * z[ (c[1],c[2]) , s ] <= x[c] * ( S[s][c[2],c[1]] + S[s][c[1],c[2]]) )
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_2, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    # Ajout du time_limit souhaité
    set_time_limit_sec(model, t_max)
    
    # Résolution du problème
    timer = @timed optimize!(model); 
    
    # Affichage si time_limit atteint
    if termination_status(model) == TIME_LIMIT
        println("Attention : time_limit atteint")
    end
    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

In [ ]:
function uncertain_model_2_ethic(C_2,V_2,C_v, S, B, logs, relax=false, t_max=120)
    """
    Crée et optimise le programme linéaire avec cycles de taille 2 augmenté de la contrainte "éthique".

    Entrées : 
        - C_2   : array   - Liste des identifiants des cycles de taille 2
        - V_2   : array   - Liste des sommets concernés par les cycles de taille 2
        - C_v   : array   - Liste des identifiants des cycles passant par chaque sommet 
        - S     : array   - Liste des scénarios
        - B     : Integer - Budget (nombre de tests maximum)
        - logs  : Boolean - Booléen pour l'affichage de la solution
        - relax : Boolean - Booléen pour relâcher la contrainte d'intégrité
        - t_max : Integer - Temps maximum accordé pour la résolution du problème
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Vecteur des sommets 
    N = Array(1:length(S))
    
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    ## x[(i,j)] = 1 ssi on réalise un test croisé sur les deux arcs (i,j) et (j,i)    
    @variable(model, x[C_2], Bin);
    ## z[(i,j),s] = 1 ssi le cycle [(i,j),(j,i)] effectue un transfert dans le scénario s
    # Relaxation de z si souhaité
    if relax
        @variable(model, z[C_2,N] >=0);
    else
        @variable(model, z[C_2,N], Bin);
    end
    
    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantations
    @objective(model, Max, 2/length(S)*sum(z[c,s] for c in C_2 for s in N));
    
    # Contraintes
    ## On réalise au plus B tests croisés (B = budget de tests)
    @constraint(model,c1, 2*sum(x[a] for a in C_2) <= B)
    ## NOUVELLE CONTRAINTE : Au moins 1 test croisé par individu
    @constraint(model,c1bis[v in V_2], sum(x[a] for a in C_v[v]) >= 1)
        
    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc (dans le scénario s)
    @constraint(model,c2[c in C_2, s in N],
        2 * z[ (c[1],c[2]) , s ] <= x[c] * ( S[s][c[2],c[1]] + S[s][c[1],c[2]]) )
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_2, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    # Ajout du time_limit si souhaité
    set_time_limit_sec(model, t_max)
    
    # Résolution du problème
    timer = @timed optimize!(model); 
    
    # Affichage si time_limit atteint
    if termination_status(model) == TIME_LIMIT
        println("Attention : time_limit atteint")
    end
    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

In [1]:
println("Modèles modifiés importés avec succès.")

Modèles modifiés importés avec succès.
